In [ ]:
import numpy as np 
import pandas as pd 
import sklearn
from sklearn.model_selection import *

In [ ]:
usa_2018_df = pd.read_csv('usa_2018_cars')

In [ ]:
usa_2018_df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
usa_car_categorical_df = usa_2018_df[['model_id','model_make_display','model_engine_compression','model_make_id','model_name','model_trim']]

In [ ]:
usa_car_numerical_df = usa_2018_df.drop(['model_make_display','model_engine_compression','model_make_id','model_name','model_trim'],axis=1)

In [ ]:
usa_car_numerical_df.info()

In [ ]:
import pandas_profiling

In [ ]:
pandas_profiling.ProfileReport(usa_car_numerical_df)

In [ ]:
usa_car_numerical_df.drop(['model_lkm_hwy','model_lkm_city'],axis=1,inplace=True)

In [ ]:

# for an approximate result, divide the speed value by 1.609
# mm to in divide the length value by 25.4
# kg to lbs for an approximate result, multiply the mass value by 2.205
# l to gal for an approximate result, divide the volume value by 3.785

In [ ]:
def lkm_to_mpg(lkm):
    return round((3.785411784*(lkm/1.609344)),1)
usa_car_numerical_df['average_mpg'] = usa_car_numerical_df['model_lkm_mixed']

In [ ]:
usa_car_numerical_df['model_horsepower'] = usa_car_numerical_df['model_engine_power_ps']

In [ ]:
def mm_to_inches(mm):
    return round(mm/25.4,1)
# usa_car_numerical_df['model_lenght_in'] = usa_car_numerical_df['model_length_mm'].apply(mm_to_inches)
# # usa_car_numerical_df['model_width_in'] = usa_car_numerical_df['model_width_mm'].apply(mm_to_inches)
# usa_car_numerical_df['model_height_in'] = usa_car_numerical_df['model_height_mm'].apply(mm_to_inches)
# usa_car_numerical_df['model_wheelbase_in'] = usa_car_numerical_df['model_wheelbase_mm'].apply(mm_to_inches)
# usa_car_numerical_df['engine_stroke_in'] = usa_car_numerical_df['model_engine_stroke_mm'].apply(mm_to_inches)
# usa_car_numerical_df['engine_bore_in'] = usa_car_numerical_df['model_engine_bore_mm'].apply(mm_to_inches)

In [ ]:
def l_to_gal(l):
    return round(l/3.785,1)
usa_car_numerical_df['fuel_cap_galons'] = usa_car_numerical_df['model_fuel_cap_l']

In [ ]:
def nm_to_ftlbs(nm):
    return round(nm*0.73756,1)
usa_car_numerical_df['engine_ft_lbs_torque'] = usa_car_numerical_df['model_engine_torque_nm'].apply(nm_to_ftlbs)

In [ ]:
usa_car_numerical_df.drop(['model_lkm_mixed','model_engine_power_ps',
                'model_fuel_cap_l',]
                ,axis=1,inplace=True)

In [ ]:
usa_car_numerical_df.drop('model_engine_cc',axis=1,inplace=True)

In [ ]:
usa_car_numerical_df.drop('model_engine_torque_nm',axis=1,inplace=True)

In [ ]:
usa_car_numerical_df.drop('model_engine_position',axis=1,inplace=True)
usa_car_numerical_df.drop('model_sold_in_us',axis=1,inplace=True)

In [ ]:
usa_car_numerical_df.set_index('model_id',inplace=True)

In [ ]:
usa_car_numerical_df.columns

In [ ]:
pandas_profiling.ProfileReport(usa_car_numerical_df)

In [ ]:
usa_car_numerical_df = pd.get_dummies(usa_car_numerical_df,columns=['model_body','model_drive',
                                                                    'model_engine_type','model_engine_fuel',
                                                                    'model_transmission_type'],drop_first=True)

In [ ]:
usa_car_numerical_df.average_mpg.argmax()

In [ ]:
usa_car_numerical_df.average_mpg.mean()

In [ ]:
usa_car_categorical_df.set_index('model_id',inplace=True)

In [ ]:
usa_car_categorical_df.query('73253')

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss = ss.fit_transform(usa_car_numerical_df)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=38, init='k-means++', n_init=10, max_iter=300, 
                tol=0.0001, precompute_distances=True,verbose=3, random_state=42, copy_x=True,
                n_jobs=None, algorithm='auto').fit(ss)

In [ ]:
args = np.argwhere(kmeans.labels_ == 36).shape

In [ ]:
args[0]

In [ ]:
def get_model_id_name(cluster):
    for num in cluster:
        x_shape = np.argwhere(kmeans.labels_ == num).shape[0]
        args = np.argwhere(kmeans.labels_ == num).reshape(x_shape,)
        print(just_model_id_name.iloc[args])
        
get_model_id_name([33,6])

In [ ]:
just_model_id_name = no_model_trim.drop(['model_make_display','model_engine_compression'],axis=1)

In [ ]:
no_model_trim.iloc[args]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

car_sparse = sparse.csr_matrix(kmeans.cluster_centers_)

similarities = cosine_similarity(car_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

#also can output sparse matrices
similarities_sparse = cosine_similarity(car_sparse,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

In [ ]:
# import csv
# pd.DataFrame(similarities).to_csv('car_cosine_vectors.tsv',sep='\t', quoting=csv.QUOTE_NONE)

In [ ]:
# import csv
# pd.DataFrame(ss).to_csv('all_cosine_vectors.tsv',sep='\t', quoting=csv.QUOTE_NONE)

In [ ]:
no_model_trim = usa_car_categorical_df.drop('model_trim',axis=1)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca_df = pca.fit_transform(ss)


In [ ]:
def scree_plot(ax, pca, n_components_to_plot=20, title=None):
    """Make a scree plot showing the variance explained (i.e. variance
    of the projections) for the principal components in a fit sklearn
    PCA object.
    
    Parameters
    ----------
    ax: matplotlib.axis object
      The axis to make the scree plot on.
      
    pca: sklearn.decomposition.PCA object.
      A fit PCA object.
      
    n_components_to_plot: int
      The number of principal components to display in the scree plot.
      
    title: str
      A title for the scree plot.
    """
    num_components = pca.n_components_
    ind = np.arange(num_components)
    vals = pca.explained_variance_ratio_
    ax.plot(ind, vals, color='blue')
    ax.scatter(ind, vals, color='blue', s=50)

    for i in range(num_components):
        ax.annotate(r"{:2.2f}%".format(vals[i]), 
        (ind[i]+0.2, vals[i]+0.005), 
        va="bottom", 
        ha="center", 
        fontsize=12)

    ax.set_xticklabels(ind, fontsize=12)
    ax.set_ylim(0, max(vals) + 0.05)
    ax.set_xlim(0 - 0.45, n_components_to_plot + 0.45)
    ax.set_xlabel("Principal Component", fontsize=12)
    ax.set_ylabel("Variance Explained (%)", fontsize=12)
    if title is not None:
        ax.set_title(title, fontsize=16)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(12,6))
scree_plot(ax, pca, n_components_to_plot=20, title='Car Matching')

In [ ]:
def get_car_back(num):
    index = usa_car_categorical_df.iloc[num]
    return index
get_car_back(1906)

In [ ]:
usa_car_categorical_df.query('76481')

In [ ]:
kmeans.fit_predict

In [ ]:
from sklearn.preprocessing import RobustScaler
robsca = RobustScaler()
robsca = robsca.fit_transform(usa_car_numerical_df)
model = KMeans(n_clusters=38,init='k-means++', max_iter=100, n_init=1,verbose=2)
model.fit(robsca)

In [ ]:
args = np.argwhere(kmeans.labels_ == 37).reshape(94,)

In [ ]:
no_model_trim.iloc[args]

In [ ]:
np.unique(no_model_trim.iloc[args])

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
print(order_centroids)

In [ ]:
print(model.cluster_centers_)

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as py
%matplotlib inline
import tqdm 

In [ ]:
silhouette_scores = []
for n in range(2,100):
    n_model = KMeans(n_clusters=n)
    n_model.fit(ss)
    silhouette_scores.append(silhouette_score(ss, n_model.labels_))

In [ ]:
py.plot(silhouette_scores)

In [ ]:
from collections import Counter 

In [ ]:
# assigned_cluster = kmeans.transform(usa_car_numerical_df).argmin(axis=1)
# for i in range(kmeans.n_clusters):
#     cluster = np.arange(0, usa_car_numerical_df.shape[0])[assigned_cluster==i]
#     topics = usa_car_categorical_df.iloc[cluster]
#     most_common = Counter(topics).most_common()
#     print("Cluster %d: %s" % (i, most_common[0][0]))
#     if len(most_common) > 1:
#         print(" %s" % (most_common[1][0]))

In [ ]:
import umap.umap_ 
import seaborn as sns 
sns.set(style='white', rc={'figure.figsize':(10,8)})

In [ ]:
standard_embedding = umap.umap_.UMAP(random_state=42).fit_transform(ss)
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], s=0.1, cmap='Spectral');

In [ ]:
kmeans_labels = KMeans(n_clusters=37).fit_predict(usa_car_numerical_df)
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=kmeans_labels, s=0.1, cmap='Spectral');

In [ ]:
import hdbscan

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, gen_min_span_tree=True)
clusterer.fit(ss)

In [ ]:
# clusterer.minimum_spanning_tree_.plot(edge_cmap='viridis',
#                                       edge_alpha=0.6,
#                                       node_size=80,
#                                       edge_linewidth=2)

In [ ]:
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [ ]:
clusterer.condensed_tree_.plot()